In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display



pd.set_option('display.max_columns', None)

import warnings

warnings.filterwarnings("ignore")

# Check County to Project
This robusness check is to view whether the projects that are built suitability score measure up to the county level suitability score. 

This lets us measure how good of a statistic is the county level suitability score when analyzing project level factors.

In [4]:
basepath = "../data/"
# Load county level suitability data
suitability_county = pd.read_csv(basepath + "suitability_scores/suitability_scores_county.csv", dtype={"GEOID": str})

# Load block group data
suitability_block_group = pd.read_csv(basepath + "suitability_scores/suitability_scores_county.csv", dtype={"GEOID": str})

# Load FIPS data
FIPS = pd.read_csv(basepath + "extras/US_FIPS_Codes.csv", dtype={"FIPS State": str, "FIPS County": str})


# Merge to obtain GEOID
suitability_county = suitability_county.merge(FIPS, left_on=["State", "County Name"], right_on=["State", "County Name"], how="left")


# Merge FIPS data to get GEOID
suitability_block_group = suitability_block_group.merge(FIPS, left_on=["State", "County Name"], right_on=["State", "County Name"], how="left")

# Load Project level suitability data
suitability_proj = pd.read_csv(basepath + "suitability_scores/suitability_scores_project.csv", dtype={"GEOID": str, 'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'BLKGRPCE': str})

In [5]:
# List of factor to check
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']

def create_analysis_df(factor, no_matches, asc = True, list_view=True, project_size='all', project_level='county'):
    if project_level == 'county':
        # Sort and filter to get the factors suitability at county level
        factor_suitability = suitability_county.sort_values(factor, ascending=asc)
        factor_suitability["STATE-COUNTY-FP"] = factor_suitability["FIPS State"] + factor_suitability["FIPS County"]
        factor_suitability = factor_suitability[['State', 'County Name', 'STATE-COUNTY-FP'] + factors].dropna()
        
        suitability_factor_proj = suitability_proj.sort_values(by=factor)[['GEOID', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'Wattage'] + factors]
        suitability_factor_proj['STATE-COUNTY-FP'] = suitability_factor_proj['STATEFP'] + suitability_factor_proj['COUNTYFP']
        
        filter_var = 'STATE-COUNTY-FP'
    elif project_level == 'bg':
            # Sort and filter to get the factors suitability at bg level
        factor_suitability = suitability_block_group.sort_values(factor, ascending=True)
        factor_suitability = factor_suitability[['State', 'County Name', 'GEOID'] + factors].dropna()
        
        suitability_factor_proj = suitability_proj[['GEOID', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'Wattage'] + factors]
        
        filter_var = 'GEOID'
    
    if project_size == 'small':
        suitability_factor_proj = suitability_factor_proj[suitability_factor_proj['Wattage'] < 5]
    elif project_size == 'medium':
        suitability_factor_proj = suitability_factor_proj[suitability_factor_proj['Wattage'] >= 5]
        suitability_factor_proj = suitability_factor_proj[suitability_factor_proj['Wattage'] < 25]
    elif project_size == 'large':
        suitability_factor_proj = suitability_factor_proj[suitability_factor_proj['Wattage'] >= 25]
    
    
    # Get the matches with teh projects
    project_matches_list = []
    for i in range(len(factor_suitability)):
        # get the state-county-fp
        state_county_fp = factor_suitability.iloc[i][filter_var]
        analysis_level_factors = factor_suitability.iloc[i][factors]
        # add _county to the factors
        analysis_level_factors = analysis_level_factors.add_suffix("_county") if project_level == 'county' else analysis_level_factors.add_suffix("_bg")
        if state_county_fp in suitability_factor_proj[filter_var].values:
            # add the rank of the project
            proj_matches = suitability_factor_proj[suitability_factor_proj[filter_var] == state_county_fp]        
            
            # Handle the list df
            proj_matches["County Rank"] = i + 1
            # get the county names and state names for the projects
            project_matches_list_df = proj_matches.merge(FIPS, left_on=["STATEFP", "COUNTYFP"], right_on=["FIPS State", "FIPS County"], how="left")
            project_matches_list_df = project_matches_list_df[['State', 'County Name', 'GEOID', 'County Rank', 'Wattage'] + factors].reset_index().drop(columns=["index"])
            project_matches_list_df[analysis_level_factors.index] = analysis_level_factors.values
            project_matches_list.append(project_matches_list_df)
            
            
        # Break when we get x matches
        if len(project_matches_list) >= no_matches:
            break
        
    if list_view:
        final =  pd.concat(project_matches_list)
        # Reorder the columns put State	County Name	GEOID	County Rank	Wattage first then followed by the factors[0] factorscounty[0] factors[1] factorscounty[1]...
        final = final[['State', 'County Name', 'GEOID', 'County Rank', 'Wattage'] + [item for sublist in zip(factors, [f + "_county" if project_level == 'county' else f + '_bg' for f in factors]) for item in sublist]]
        return final

In [6]:
dropdown = widgets.Dropdown(
    options=factors,
    description='Select DF:'
)

dropdown_project_size = widgets.Dropdown(
    options=['all', 'small', 'medium', 'large'],
    description='Select Project Size:'
)

# add an input for selecting an integer
no_matches = widgets.IntText(
    value=10,
    description='No of Matches:',
    disabled=False
)

dropdown_project_level = widgets.Dropdown(
    options=['county', 'bg'],
    description='Select Project Level:'
)

# Create a button to trigger the display
button = widgets.Button(description="Display")
button_download = widgets.Button(description="Download to CSV")
output = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'width': "2400px", 'overflow_y': 'scroll'})

def on_button_clicked(b):
    with output:
        output.clear_output()
        display(create_analysis_df(dropdown.value, no_matches.value, project_size=dropdown_project_size.value, project_level=dropdown_project_level.value))
        
def on_button_download_clicked(b):
    with output:
        output.clear_output()
        create_analysis_df(dropdown.value, no_matches.value, project_size=dropdown_project_size.value, project_level=dropdown_project_level.value).to_csv("output.csv")
        print("Downloaded to output.csv")
        
button.on_click(on_button_clicked)
button_download.on_click(on_button_download_clicked)

display(dropdown)
display(no_matches)
display(dropdown_project_size)
display(dropdown_project_level)
display(button, button_download)
display(output)

Dropdown(description='Select DF:', options=('GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density',…

IntText(value=10, description='No of Matches:')

Dropdown(description='Select Project Size:', options=('all', 'small', 'medium', 'large'), value='all')

Dropdown(description='Select Project Level:', options=('county', 'bg'), value='county')

Button(description='Display', style=ButtonStyle())

Button(description='Download to CSV', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…